# Guide to Explore Hanoi

## I. Introduction

As someone like to travel, it's a good idea to do a research beforehand for a new destination. This time I choose Hanoi, our country's capital city.

I will need to find out about where to stay, where to spend my time and... where to go if I have problem with my health. With all those infos, my trip will be exciting and safe for sure.

## II. Data Source and Tools

The data will come from:
- OpenStreetMap (OSM): to get map data, drawing district boundary, get coordinates...
- FourSquare: for exporing the city. Mainly I'm interested in **Hotel**, **cafe**, **restaurant** and **hospital**
- File *hanoi_district.csv*: it contains list of main districts of Hanoi with coordinates (the coordinates will be filled later in this notebook)

I will also use the online tool from https://tyrasd.github.io/osmtogeojson/ to convert OSM data into geojson for use with **folium**

## III. Build a Map for Hanoi's main districts

## Import library

In [ ]:
import numpy as np
import pandas as pd
import io
import requests

## Begin build map

We already have the list of main district of Hanoi in a CSV file, so we will load the data

In [ ]:
with io.open('hanoi_district.csv','r') as file:
    df = pd.read_csv(file)

print('There are', df.shape[0], 'main district in Hanoi')
df

We will use OpenStreetMap to get the info for district boundary. First we will build the query

## Build the query for all the district at once

Define the function to get boundary data

In [ ]:
def get_OSM_data(POI_list, to_file=False, filename='result.xml'):
    query_head = """
    [out:xml][timeout:25];
    (\n"""

    query_body = ''
    for d in POI_list:
        query_body += """relation["name:en"=\""""+d+"""\"]; (._;>;);\n"""

    query_tail = """);
    out center;
    """
    
    query = (query_head+query_body+query_tail)

    overpass_url = "http://overpass-api.de/api/interpreter"
    
    response = requests.get(overpass_url, params={'data': query})

    if to_file:
        with io.open(filename,'w+', encoding='utf8') as file:
            file.write(response.text)

    return response.text

Query OpenStreetMap for boundary of district.

In [ ]:
data = get_OSM_data(df['Name'], True, 'hanoi_districts.xml')

With the XML file, we can use the online tool at https://tyrasd.github.io/osmtogeojson/ to convert it to geojson file. We named it ***hanoi_districts.geojson***

## Draw map using geojson data from our *result.geojson* file

Get the center point of Hanoi using OSM, save the data to file to user later if needed

In [ ]:
data = get_OSM_data(['Hanoi'], True, 'hanoi.xml')

Use BeautifulSoup to extract the center point. Since we know Hanoi has **admin_level = 2**, we will only get the coordinates of the node that satisfied the condition

In [ ]:
from bs4 import BeautifulSoup

nodes = BeautifulSoup(data, 'lxml').findAll('node')

for node in nodes:
    if node.find('tag', {'k':'admin_level', 'v':2}) != None:
        latitude = node['lat']
        longitude = node['lon']
        break

Draw map

In [ ]:
import folium

map = folium.Map(location=[latitude, longitude], width='100%', height=400, zoom_start=11)

# We use io.open since the normal method won't work reliability with utf-8 encode file
with io.open('hanoi_districts.geojson','r', encoding='utf8') as file:
    geojson = file.read()

folium.GeoJson(geojson, name='District boundaries').add_to(map)

# Add marker
#folium.CircleMarker([latitude, longitude], radius=5,color='blue',fill=True,
#                       fill_color='#3186cc', fill_opacity=0.7,
#                       parse_html=False).add_to(map)

#folium.LayerControl().add_to(map)

map

In [ ]:
inf = BeautifulSoup(data,'lxml')

In [ ]:
inf.find("node")['lat']